In [ ]:
"""
LORA Data Processing Script (ROOT → DAT, H5, NUR)

This script processes raw LORA .root files and converts them into multiple output formats:

- .dat files:
    Human-readable summary of reconstructed air-shower and detector-level 
    parameters. These files are text-based and can be easily inspected 
    for quick interpretation of event properties (core position, arrival 
    direction, energy estimates, detector signals, etc.).

- .h5 files:
    HDF5 containers storing ADC traces (counts per detector). These are 
    structured binary files optimized for handling large numerical arrays 
    and can be used for further analysis or machine learning pipelines.

- .nur files:
    Binary event files in the NuRadioReco format. These files store the 
    reconstructed shower and detector information in a standardized 
    framework suitable for subsequent analysis with NuRadioReco modules.

Main Steps:
1. Read ROOT files containing raw LORA event data.
2. Apply detector calibrations, noise correction, and signal reconstruction.
3. Reconstruct air-shower parameters (arrival direction, core, energy, etc.).
4. Write results to:
   - `.dat` (human-readable text),
   - `.h5` (ADC traces),
   - `.nur` (NuRadioReco-compatible binary format).

Functions :
- write_nur(evt, detectors, ev, outputdir, run_number, event_id):
    Converts a reconstructed event into a NuRadioReco .nur file.
- ProcessEventV2(outputdir, data_dir, root_name, run_number=0): Taken from the scripts of Lucas Van Dongen
    Full processing chain for a single ROOT file.

Usage:
    Adjust the `date`, `data_dir`, and `outputdir` variables to match 
    your dataset and desired output location. The script automatically 
    finds and processes all matching ROOT files for the given date.
"""


import numpy as np
import glob, os, sys
import matplotlib.pyplot as plt
import datetime
from datetime import datetime


import ROOT

import LORAparameters as LORA
import process_functions
import read_functions as read
import detector
import event
import h5py

# NuRadioReco imports for .nur
from NuRadioReco.framework import event as nr_event, hybrid_station, hybrid_channel, hybrid_shower
from NuRadioReco.framework import parameters
from NuRadioReco.modules.io import eventWriter
from NuRadioReco.utilities import units


def write_nur(evt, detectors, ev, outputdir, run_number, event_id):
    print(f"[write_nur] Called for event_id={event_id}, event_flag={ev.event_flag}")
    if ev.event_flag != 0:
        print(f"[write_nur] Skipping .nur file for flagged event {event_id}")
        return

    lora_air_shower = hybrid_shower.HybridShower('LORA')
    lora_air_shower.set_parameter(parameters.showerParameters.core, (ev.x_core, ev.y_core, 0.0))
    lora_air_shower.set_parameter(parameters.showerParameters.zenith, 90.0 - ev.fit_elevation)
    lora_air_shower.set_parameter(parameters.showerParameters.azimuth, ev.fit_phi * units.deg)
    lora_air_shower.set_parameter(parameters.showerParameters.energy, ev.energy * 1e15)
    evt.add_shower(lora_air_shower)

    station = hybrid_station.HybridStation(station_id=0)
    for i, det in enumerate(detectors):
        channel = hybrid_channel.Channel(channel_id=i)
        channel.set_parameter(parameters.hybridChannelParameters.signal_time, float(det.gps))
        channel.set_parameter(parameters.hybridChannelParameters.particle_density, float(det.density))
        channel.set_parameter(parameters.hybridChannelParameters.trigger, int(det.trig))
        station.add_channel(channel=channel)
    evt.set_particle_station(station=station)

    nur_filename = os.path.join(outputdir, f'lora_run{run_number}_{event_id}.nur')
    print(f"[write_nur] Writing to {nur_filename}")
    writer = eventWriter.eventWriter()
    writer.begin(nur_filename)
    writer.run(evt)  # <-- This is likely where the 'has_sim_station' check happens
    writer.end()
    print(f"[write_nur] .nur file written to {nur_filename}")


def ProcessEventV2(outputdir, data_dir, root_name, run_number=0):
    print(f"[ProcessEventV2] Processing {root_name}")
    time_code = int(root_name[:8])
    if time_code <= 20191116:
        timestamps, ns_timestamps = read.old_file_all_events_in_root(root_name, data_dir, minimum_dets=9)
    else:
        timestamplist, ns_timestamplist, n_active_dets, _ = read.all_events_in_root(root_name, data_dir)
        mask = n_active_dets >= 9
        timestamps = timestamplist[mask]
        ns_timestamps = ns_timestamplist[mask]

    for j, timestamp in enumerate(timestamps):
        ns_timestamp = ns_timestamps[j]
        if time_code <= 20191116:
            n_det = LORA.nDetA
            n_lasa = LORA.nLasaA
            version = 'V1'
        else:
            n_det = LORA.nDetB
            n_lasa = LORA.nLasaB
            version = 'V2'

        detectors = [detector.Detector(f'Det{i+1}') for i in range(n_det)]
        lasas = [detector.Lasa(f'Lasa{i+1}') for i in range(n_lasa)]

        detector.load_positions(detectors)
        detector.load_signal(detectors)
        detector.load_gain(detectors)

        LOFAR_id = int(timestamp) - LORA.event_id_offset
        ev = event.Event(LOFAR_id, version)
        dt_obj = datetime.utcfromtimestamp(timestamp)

        if version == 'V1':
            log_info = read.log_file(root_name, data_dir)
            info = read.return_root(root_name, timestamp, ns_timestamp, data_dir)
            sec0, sec1, sec2 = read.return_second_data(root_name, timestamp, ns_timestamp, data_dir)
            log_data = read.return_log_data(root_name, timestamp, ns_timestamp, data_dir)
            noise = read.return_noise_data(root_name, timestamp, ns_timestamp, data_dir)
            detector.load_sec_information(sec0, sec1, sec2, lasas, version)
        else:
            log_info = read.log_file(root_name, data_dir)
            info = read.return_rootV2(root_name, timestamp, ns_timestamp, data_dir)
            sec0, sec1, sec2 = read.return_second_dataV2(root_name, timestamp, ns_timestamp, data_dir)
            log_data = read.return_log_dataV2(root_name, timestamp, ns_timestamp, data_dir)
            noise = read.return_noise_dataV2(root_name, timestamp, ns_timestamp, data_dir)
            detector.load_sec_informationV2(sec0, sec1, sec2, lasas, version)

        detector.load_event_information(info, detectors)
        detector.load_log_information(log_data, detectors)
        detector.load_noise_information(noise, detectors)

        for d in detectors:
            event.find_counts(d)
            event.get_arrival_time(d)
            if version == 'V1':
                event.get_event_timestamp(d, lasas[(d.number-1)//4])
            else:
                event.get_event_timestamp_V2(d, lasas[(d.number-1)//4])

        for l in lasas:
            event.cal_event_timestamp(detectors, l)

        event.do_arrival_time_diff(detectors)
        event.do_arrival_direction(detectors, ev)
        event.do_COM_core(detectors, ev)
        event.find_density(detectors, ev)
        event.fit_arrival_direction_trial(detectors, ev)
        event.NKG_fit_trial(detectors, ev)

        print(f"[ProcessEventV2] Event {j}: event_flag={ev.event_flag}")
        if ev.event_flag == 0:
            dat_name = f"LORAdata-{dt_obj.strftime('%Y%m%dT%H%M%S')}.dat"
            with open(os.path.join(outputdir, dat_name), 'w') as f:
                f.write('//UTC_Time(secs)\tnsecs\t\tCore(X)\t\tCore(Y)\t\tElevation\tAzimuth\t\tEnergy(eV)\tCoreE(X)\tCoreE(Y)\tMoliere_rad(m)\t\trM_error\tElevaErr\tAziErr\tEnergyErr(eV)\tNe\t\tNeErr\t\tCorCoef_XY\t\tNe_RefA\t\tNeErr_RefA\t\tEnergy_RefA\t\tEnergyErr_RefA\n')
                f.write('{0}\t{1}\t{2:.2f}\t{3:.2f}\t{4:.2f}\t{5:.2f}\t{6}\t{7:.2f}\t{8:.2f}\t{9:.2f}\t{10:.2f}\t{11:.2f}\t{12}\t{13}\t{14}\t{15}\t{16}\t{17}\t{18}\t{19}\t{20}\n'.format(
                    int(ev.UTC_min), int(ev.nsec_min/10), ev.x_core, ev.y_core,
                    ev.fit_elevation, ev.fit_phi, int(ev.energy*1e15),
                    ev.x_core_err, ev.y_core_err, ev.Rm, ev.rM_err,
                    ev.fit_elevation_err, ev.fit_phi_err, int(ev.energy_err*1e15),
                    ev.Ne, ev.Ne_err, ev.CorCoef_xy, ev.Ne_RefA,
                    ev.NeErr_RefA, int(ev.Energy_RefA*1e15), int(ev.EnergyErr_RefA*1e15)
                ))
                f.write('//Detector coordinates w.r.t CS002 LBA center\n')
                f.write('//Det_no.    X_Cord(m)    Y_Cord(m)    Z_Cord(m)    UTC_time    (10*nsecs)  int_ADC_count  gain(ADC/muon)    Particle_Density(/m2), baseline(ADC), rms(ADC), corrected_peak(ADC), corrected_threshold(ADC), avg_mean(ADC), avg_sigma(ADC), trigger?\n')
                for i, det in enumerate(detectors):
                    f.write('{0}\t{1}\t{2}\t{3}\t{4}\t{5}\t{6:.4f}\t{7}\t{8:.4f}\t{9:.2f}\t{10:.2f}\t{11:.2f}\t{12:.2f}\t{13:.2f}\t{14:.2f}\t{15}\n'.format(
                        i+1, det.x_cord, det.y_cord, det.z_cord,
                        int(det.gps), int(det.cal_time), det.trace_int_counts,
                        det.gain, det.density, det.trace_mean, det.trace_rms,
                        det.corrected_peak, det.corrected_threshold,
                        det.sec_mean, det.sec_sigma, det.trig
                    ))

        counts = np.stack([d.counts for d in detectors], axis=1)
        h5_name = 'counts' + dt_obj.strftime('-%Y%m%dT%H%M%S.h5')
        with h5py.File(os.path.join(outputdir, h5_name), 'w') as hdf:
            hdf.create_dataset('counts', data=counts, dtype='int')

        # Skipping .nur file writing
        print(f"[ProcessEventV2] Calling write_nur for event {j}")
        evt = nr_event.Event(run_number=run_number, event_id=j)
        write_nur(evt, detectors, ev, outputdir, run_number, j)
        print(f"[ProcessEventV2] write_nur finished for event {j}")

# --- The rest of your notebook code for file searching and processing ---

# Define the date and other parameters
date = '20211101'  # YYYYMMDD format
run_number = 0
data_dir = '/home/wecapstor3/capn/mppi138h/LOFAR/data/LORA/LORAraw/'
outputdir = '/home/wecapstor3/capn/mppi138h/LOFAR/scratch/stuti/lora-datapipeline/2021_Nov_output/'

# Search for .root files matching the date
pattern = os.path.join(data_dir, f"{date}*.root")
file_list = glob.glob(pattern)

# Process each matching file
if not file_list:
    print(f"No .root files found for date {date} in {data_dir}")
else:
    for file_path in file_list:
        root_name = os.path.basename(file_path).replace('.root', '')
        try:
            print(f"Processing file: {root_name}")
            ProcessEventV2(outputdir, data_dir, root_name, run_number)
        except Exception as e:
            print(f"Error processing {root_name}: {e}")

print(f"Processing complete for date {date}")


Processing file: 20211101_0030
[ProcessEventV2] Processing 20211101_0030
can't find log file
reading root file: /home/wecapstor3/capn/mppi138h/LOFAR/data/LORA/LORAraw/20211101_0030.root


/tmp/ipykernel_2238791/2045337894.py:88: DeprecationWarning: datetime.datetime.utcfromtimestamp() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.fromtimestamp(timestamp, datetime.UTC).
  dt_obj = datetime.utcfromtimestamp(timestamp)


number 21,counts 0
0.0 18.5
flagged event
number 22,counts 0
0.0 18.5
flagged event
number 23,counts 0
0.0 18.5
flagged event
number 24,counts 0
0.0 18.5
flagged event
number 25,counts 0
0.0 18.5
flagged event
number 26,counts 0
0.0 18.5
flagged event
number 27,counts 0
0.0 18.5
flagged event
number 28,counts 0
0.0 18.5
flagged event
number 29,counts 0
0.0 18.5
flagged event
number 30,counts 0
0.0 18.5
flagged event
number 31,counts 0
0.0 18.5
flagged event
number 32,counts 0
0.0 18.5
flagged event
number 33,counts 0
0.0 18.5
flagged event
number 34,counts 0
0.0 18.5
flagged event
number 35,counts 0
0.0 18.5
flagged event
number 36,counts 0
0.0 18.5
flagged event
number 37,counts 0
0.0 18.5
flagged event
number 38,counts 0
0.0 18.5
flagged event
number 39,counts 0
0.0 18.5
flagged event
number 40,counts 0
0.0 18.5
flagged event
trigger condition:  3.0
sorted: [19175. 20000. 20200. 20375.]
V1:  [20200. 20000. 19175. 20375.] 20200.0
5 8755752109.0
6 8755751389.0
7 8755750787.0
8 87557517

/home/wecapstor3/capn/mppi138h/LOFAR/scratch/stuti/NuRadioMC/ROOT_to_nur_LORA/event.py:1280: RuntimeWarning: invalid value encountered in arcsin
  theta = np.arcsin(np.sqrt(l**2 + m**2)) * (180.0 / np.pi)
/home/wecapstor3/capn/mppi138h/LOFAR/scratch/stuti/NuRadioMC/ROOT_to_nur_LORA/event.py:1290: RuntimeWarning: divide by zero encountered in scalar divide
  err_theta = np.sqrt((l * err_l / n)**2 + (m * err_m / n)**2) / np.tan(theta * np.pi / 180.0)


[ProcessEventV2] Calling write_nur for event 0
[write_nur] Called for event_id=0, event_flag=0
[write_nur] Writing to /home/wecapstor3/capn/mppi138h/LOFAR/scratch/stuti/lora-datapipeline/2021_Nov_output/lora_run0_0.nur
[write_nur] .nur file written to /home/wecapstor3/capn/mppi138h/LOFAR/scratch/stuti/lora-datapipeline/2021_Nov_output/lora_run0_0.nur
[ProcessEventV2] write_nur finished for event 0
can't find log file
reading root file: /home/wecapstor3/capn/mppi138h/LOFAR/data/LORA/LORAraw/20211101_0030.root
number 21,counts 0
0.0 18.5
number 22,counts 0
0.0 18.5
number 23,counts 0
0.0 18.5
number 24,counts 0
0.0 18.5
number 25,counts 0
0.0 18.5
number 26,counts 0
0.0 18.5
number 27,counts 0
0.0 18.5
number 28,counts 0
0.0 18.5
number 29,counts 0
0.0 18.5
number 30,counts 0
0.0 18.5
number 31,counts 0
0.0 18.5
number 32,counts 0
0.0 18.5
number 33,counts 0
0.0 18.5
number 34,counts 0
0.0 18.5
number 35,counts 0
0.0 18.5
number 36,counts 0
0.0 18.5
number 37,counts 0
0.0 18.5
number 38,

In [ ]:
#Code to open a .nur file

import NuRadioReco.modules.io.eventReader
from NuRadioReco.framework import parameters

event_reader = NuRadioReco.modules.io.eventReader.eventReader()
event_reader.begin([
    '/home/wecapstor3/capn/mppi138h/LOFAR/scratch/stuti/lora-datapipeline/2021_Nov_output/lora_run0_47.nur'
])

for event in event_reader.run():
    station = event.get_particle_station(0)

    for channel in station.iter_channels():
        signal_time = channel.get_parameter(parameters.hybridChannelParameters.signal_time)
        particle_density = channel.get_parameter(parameters.hybridChannelParameters.particle_density)
        triggered = channel.get_parameter(parameters.hybridChannelParameters.trigger)

        print(f"Signal time: {signal_time}")
        print(f"Particle density: {particle_density}")
        print(f"Triggered: {triggered}")

    hybrid_info = event.get_hybrid_information()
    for shower in hybrid_info.get_hybrid_showers():
        core = shower.get_parameter(parameters.showerParameters.core)
        zenith = shower.get_parameter(parameters.showerParameters.zenith)
        azimuth = shower.get_parameter(parameters.showerParameters.azimuth)
        energy = shower.get_parameter(parameters.showerParameters.energy)

        print(f"Core: {core}")
        print(f"Zenith: {zenith}")
        print(f"Azimuth: {azimuth}")
        print(f"Energy: {energy}")


Signal time: 0.0
Particle density: 0.0
Triggered: 0
Signal time: 0.0
Particle density: 0.0
Triggered: 0
Signal time: 0.0
Particle density: 0.0
Triggered: 0
Signal time: 0.0
Particle density: 0.0
Triggered: 0
Signal time: 1635958030.0
Particle density: 51.26585050848367
Triggered: 1
Signal time: 1635958030.0
Particle density: 35.00851996240738
Triggered: 1
Signal time: 1635958030.0
Particle density: 7.82935037792215
Triggered: 1
Signal time: 1635958030.0
Particle density: 6.826971776567179
Triggered: 0
Signal time: 0.0
Particle density: 0.0
Triggered: 0
Signal time: 0.0
Particle density: 0.0
Triggered: 0
Signal time: 0.0
Particle density: 0.0
Triggered: 0
Signal time: 0.0
Particle density: 0.0
Triggered: 0
Signal time: 1635958030.0
Particle density: 28.002018144931643
Triggered: 1
Signal time: 1635958030.0
Particle density: 22.674170849187583
Triggered: 1
Signal time: 1635958030.0
Particle density: 80.761175110209
Triggered: 1
Signal time: 1635958030.0
Particle density: 88.5055200134345